# Week 3: SQL Basics for Marketing Analytics

**Goal:** Master SQL fundamentals to query and analyze marketing data directly from databases.

**Time Commitment:** ~1 hour per day × 7 days = 7 hours total

**What You'll Learn:**
- SQL SELECT statements and WHERE clauses
- JOIN operations to combine tables
- GROUP BY and aggregations for metrics
- Subqueries for complex analysis
- CASE statements for conditional logic
- Window functions for advanced calculations
- Real marketing database queries

**Why This Matters:**
As a Marketing Measurement Partner, you'll query data from:
- Campaign management platforms
- CRM systems
- Data warehouses (BigQuery, Snowflake, Redshift)
- Attribution databases

SQL is the universal language for data retrieval. While Pandas is great for analysis, SQL is essential for:
- Extracting data from production databases
- Working with large datasets (millions/billions of rows)
- Collaborating with data engineering teams
- Building data pipelines

---

## Setup: Create Sample Marketing Database

We'll use SQLite to create a sample marketing database with realistic tables.

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Create database connection
conn = sqlite3.connect(':memory:')  # In-memory database
cursor = conn.cursor()

print("✅ Marketing database created successfully!")

In [ ]:
# Create campaigns table
cursor.execute('''
CREATE TABLE campaigns (
    campaign_id INTEGER PRIMARY KEY,
    campaign_name TEXT NOT NULL,
    channel TEXT NOT NULL,
    campaign_type TEXT,
    start_date DATE,
    end_date DATE,
    budget REAL,
    status TEXT
)
''')

# Insert sample campaigns
campaigns_data = [
    (1, 'Search Brand - Q4', 'Google', 'Search', '2024-10-01', '2024-12-31', 50000, 'active'),
    (2, 'Search Generic - Q4', 'Google', 'Search', '2024-10-01', '2024-12-31', 75000, 'active'),
    (3, 'Facebook Prospecting', 'Meta', 'Social', '2024-09-01', '2024-12-31', 60000, 'active'),
    (4, 'Facebook Retargeting', 'Meta', 'Social', '2024-09-01', '2024-12-31', 40000, 'active'),
    (5, 'Display Remarketing', 'Google', 'Display', '2024-10-01', '2024-12-31', 30000, 'active'),
    (6, 'Instagram Stories', 'Meta', 'Social', '2024-10-15', '2024-12-31', 35000, 'active'),
    (7, 'LinkedIn B2B', 'LinkedIn', 'Social', '2024-11-01', '2024-12-31', 45000, 'active'),
    (8, 'TikTok Awareness', 'TikTok', 'Video', '2024-10-01', '2024-11-30', 25000, 'paused')
]

cursor.executemany('INSERT INTO campaigns VALUES (?,?,?,?,?,?,?,?)', campaigns_data)

print("✅ Campaigns table created with 8 campaigns")

In [ ]:
# Create daily_performance table
cursor.execute('''
CREATE TABLE daily_performance (
    performance_id INTEGER PRIMARY KEY,
    campaign_id INTEGER,
    date DATE,
    impressions INTEGER,
    clicks INTEGER,
    conversions INTEGER,
    cost REAL,
    revenue REAL,
    FOREIGN KEY (campaign_id) REFERENCES campaigns(campaign_id)
)
''')

# Generate sample daily performance data
np.random.seed(42)
performance_data = []
perf_id = 1

for campaign_id in range(1, 9):
    for day in range(30):  # 30 days of data per campaign
        date = (datetime(2024, 10, 1) + timedelta(days=day)).strftime('%Y-%m-%d')
        
        # Different campaigns have different performance profiles
        if campaign_id in [1, 2]:  # Search campaigns
            impressions = np.random.randint(3000, 8000)
            clicks = np.random.randint(150, 400)
            conversions = np.random.randint(8, 25)
        elif campaign_id in [3, 4, 6]:  # Social campaigns
            impressions = np.random.randint(15000, 40000)
            clicks = np.random.randint(500, 1500)
            conversions = np.random.randint(10, 35)
        else:  # Display/Video
            impressions = np.random.randint(25000, 60000)
            clicks = np.random.randint(300, 900)
            conversions = np.random.randint(5, 20)
        
        cost = np.random.uniform(100, 500)
        revenue = cost * np.random.uniform(2.5, 5.0)
        
        performance_data.append((perf_id, campaign_id, date, impressions, clicks, conversions, cost, revenue))
        perf_id += 1

cursor.executemany('INSERT INTO daily_performance VALUES (?,?,?,?,?,?,?,?)', performance_data)

print(f"✅ Daily performance table created with {len(performance_data)} rows")

In [ ]:
# Create customers table
cursor.execute('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    signup_date DATE,
    customer_segment TEXT,
    ltv REAL
)
''')

# Sample customer data
customers_data = [
    (1, 'John', 'Smith', 'john.smith@email.com', '2024-10-01', 'high_value', 1250.00),
    (2, 'Sarah', 'Johnson', 'sarah.j@email.com', '2024-10-02', 'medium_value', 450.00),
    (3, 'Michael', 'Williams', 'mike.w@email.com', '2024-10-03', 'high_value', 1800.00),
    (4, 'Emily', 'Brown', 'emily.b@email.com', '2024-10-03', 'low_value', 150.00),
    (5, 'David', 'Jones', 'david.j@email.com', '2024-10-05', 'medium_value', 620.00)
]

cursor.executemany('INSERT INTO customers VALUES (?,?,?,?,?,?,?)', customers_data)

print("✅ Customers table created with 5 customers")

In [ ]:
# Create conversions table (attribution)
cursor.execute('''
CREATE TABLE conversions (
    conversion_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    campaign_id INTEGER,
    conversion_date DATE,
    conversion_value REAL,
    conversion_type TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (campaign_id) REFERENCES campaigns(campaign_id)
)
''')

conversions_data = [
    (1, 1, 1, '2024-10-01', 250.00, 'purchase'),
    (2, 2, 3, '2024-10-02', 150.00, 'purchase'),
    (3, 3, 2, '2024-10-03', 450.00, 'purchase'),
    (4, 4, 4, '2024-10-03', 75.00, 'purchase'),
    (5, 1, 4, '2024-10-08', 300.00, 'repeat_purchase'),
    (6, 5, 3, '2024-10-05', 220.00, 'purchase'),
    (7, 3, 1, '2024-10-12', 380.00, 'repeat_purchase')
]

cursor.executemany('INSERT INTO conversions VALUES (?,?,?,?,?,?)', conversions_data)

print("✅ Conversions table created with 7 conversions")
print("\n✅ Marketing database setup complete! Ready to learn SQL.")

## 📅 Day 15: SQL Intro, SELECT, WHERE (~60 min)

### Learning Objectives
- Understand SQL syntax and structure
- Write SELECT statements to retrieve data
- Use WHERE clauses to filter results
- Apply comparison operators

### The Business Problem
You need to extract specific campaign data from your marketing database to answer questions like:
- Which campaigns are currently active?
- What's the budget for Google campaigns?
- Which campaigns started in October?

### 📖 Concept: The SELECT Statement

SELECT is the foundation of SQL - it retrieves data from tables.

**Basic syntax:**
```sql
SELECT column1, column2, ...
FROM table_name;
```

In [ ]:
# Select all columns from campaigns
query = '''
SELECT *
FROM campaigns;
'''

result = pd.read_sql_query(query, conn)
print("All Campaigns:")
print(result)

In [ ]:
# Select specific columns
query = '''
SELECT campaign_name, channel, budget
FROM campaigns;
'''

result = pd.read_sql_query(query, conn)
print("Campaign Summary:")
print(result)

### 📖 Concept: WHERE Clause

WHERE filters rows based on conditions.

In [ ]:
# Filter for active campaigns
query = '''
SELECT campaign_name, channel, status
FROM campaigns
WHERE status = 'active';
'''

result = pd.read_sql_query(query, conn)
print("Active Campaigns:")
print(result)

In [ ]:
# Filter with numeric comparison
query = '''
SELECT campaign_name, budget
FROM campaigns
WHERE budget > 40000;
'''

result = pd.read_sql_query(query, conn)
print("High Budget Campaigns (> $40,000):")
print(result)

### 📖 Concept: Multiple Conditions (AND, OR)

Combine conditions for more specific filtering.

In [ ]:
# AND: Both conditions must be true
query = '''
SELECT campaign_name, channel, budget
FROM campaigns
WHERE channel = 'Google' AND budget > 40000;
'''

result = pd.read_sql_query(query, conn)
print("High-Budget Google Campaigns:")
print(result)

In [ ]:
# OR: At least one condition must be true
query = '''
SELECT campaign_name, channel
FROM campaigns
WHERE channel = 'Meta' OR channel = 'LinkedIn';
'''

result = pd.read_sql_query(query, conn)
print("Social Media Campaigns:")
print(result)

### 📖 Concept: IN Operator

Check if a value matches any in a list.

In [ ]:
# More elegant way to check multiple values
query = '''
SELECT campaign_name, channel
FROM campaigns
WHERE channel IN ('Meta', 'LinkedIn', 'TikTok');
'''

result = pd.read_sql_query(query, conn)
print("Social Media Campaigns (using IN):")
print(result)

### ✏️ Exercise 1: Filter Campaigns

Write queries to answer these questions:

In [ ]:
# YOUR CODE HERE
# 1. Find all Search campaigns (campaign_type = 'Search')

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print(result)

In [ ]:
# YOUR CODE HERE
# 2. Find campaigns with budget between 30000 and 50000
# Hint: Use BETWEEN or >= and <=

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print(result)

In [ ]:
# YOUR CODE HERE
# 3. Find active Google campaigns with budget > 30000

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print(result)

### 🎯 Day 15 Mini-Project: Campaign Audit Query

Write a query that identifies campaigns needing review based on multiple criteria.

In [ ]:
# YOUR CODE HERE
# Find campaigns that are:
# - Active
# - Started before November 1, 2024
# - Have budget >= 35000
# - Are either Google or Meta channels
#
# Display: campaign_name, channel, budget, start_date

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Campaigns for Review:")
# print(result)

### 🎓 Day 15 Key Takeaways

✅ SELECT retrieves data from tables  
✅ WHERE filters rows based on conditions  
✅ Use AND/OR to combine conditions  
✅ IN operator checks against multiple values  
✅ SQL is case-insensitive but UPPERCASE keywords aid readability  

**Next:** Tomorrow we'll learn JOINs to combine data from multiple tables!

---

## 📅 Day 16: JOIN Operations (~60 min)

### Learning Objectives
- Understand different types of JOINs
- Combine campaign and performance data
- Link customers to conversions
- Master INNER, LEFT, RIGHT, and OUTER joins

### The Business Problem
Data is spread across multiple tables:
- Campaign metadata (campaigns table)
- Daily performance (daily_performance table)
- Customer info (customers table)
- Conversion events (conversions table)

JOINs let you combine these for comprehensive analysis.

### 📖 Concept: INNER JOIN

Returns rows where there's a match in both tables.

In [ ]:
# Join campaigns with daily performance
query = '''
SELECT 
    c.campaign_name,
    c.channel,
    dp.date,
    dp.cost,
    dp.conversions,
    dp.revenue
FROM campaigns c
INNER JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
LIMIT 10;
'''

result = pd.read_sql_query(query, conn)
print("Campaign Performance Data (First 10 rows):")
print(result)

### 📖 Concept: Multiple Table JOINs

Join three or more tables together.

In [ ]:
# Join customers, conversions, and campaigns
query = '''
SELECT 
    cu.first_name,
    cu.last_name,
    cu.customer_segment,
    ca.campaign_name,
    ca.channel,
    co.conversion_date,
    co.conversion_value,
    co.conversion_type
FROM customers cu
INNER JOIN conversions co ON cu.customer_id = co.customer_id
INNER JOIN campaigns ca ON co.campaign_id = ca.campaign_id;
'''

result = pd.read_sql_query(query, conn)
print("Customer Conversion Attribution:")
print(result)

### 📖 Concept: LEFT JOIN

Returns all rows from left table, matched rows from right table (or NULL if no match).

In [ ]:
# Get all customers, whether they converted or not
query = '''
SELECT 
    cu.customer_id,
    cu.first_name,
    cu.last_name,
    COUNT(co.conversion_id) as conversion_count
FROM customers cu
LEFT JOIN conversions co ON cu.customer_id = co.customer_id
GROUP BY cu.customer_id, cu.first_name, cu.last_name;
'''

result = pd.read_sql_query(query, conn)
print("All Customers with Conversion Counts:")
print(result)

### ✏️ Exercise 2: Practice JOINs

Write queries using JOINs.

In [ ]:
# YOUR CODE HERE
# 1. Join campaigns and daily_performance
# Show: campaign_name, channel, date, clicks, conversions
# Filter for only Google campaigns
# Limit to 15 rows

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print(result)

In [ ]:
# YOUR CODE HERE
# 2. Find all conversions with customer names and campaign names
# Show: customer name, campaign_name, conversion_value, conversion_type
# Sort by conversion_value descending

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print(result)

### 🎯 Day 16 Mini-Project: Campaign Performance Report

Create a comprehensive report joining multiple tables.

In [ ]:
# YOUR CODE HERE
# Create a report showing:
# - Campaign name and channel
# - Total impressions, clicks, conversions
# - Total cost and revenue
# - Calculated CTR (clicks/impressions)
# - Calculated ROAS (revenue/cost)
#
# Join campaigns and daily_performance
# Group by campaign
# Sort by ROAS descending

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Campaign Performance Summary:")
# print(result)

### 🎓 Day 16 Key Takeaways

✅ INNER JOIN returns only matching rows  
✅ LEFT JOIN keeps all left table rows  
✅ Can join multiple tables in one query  
✅ Use table aliases (c, dp) for readability  
✅ JOIN conditions use ON clause  

**Next:** Tomorrow we'll master GROUP BY and aggregations!

---

## 📅 Day 17: GROUP BY and Aggregations (~60 min)

### Learning Objectives
- Use GROUP BY to aggregate data
- Apply aggregate functions (SUM, AVG, COUNT, MIN, MAX)
- Filter grouped data with HAVING
- Calculate marketing metrics at different levels

### The Business Problem
You need to analyze performance by:
- Channel (Google vs Meta vs LinkedIn)
- Campaign type (Search vs Social vs Display)
- Time period (weekly, monthly)
- Customer segment

### 📖 Concept: Basic GROUP BY

GROUP BY collapses rows with the same value into summary rows.

In [ ]:
# Total budget by channel
query = '''
SELECT 
    channel,
    COUNT(*) as campaign_count,
    SUM(budget) as total_budget,
    AVG(budget) as avg_budget
FROM campaigns
GROUP BY channel
ORDER BY total_budget DESC;
'''

result = pd.read_sql_query(query, conn)
print("Budget by Channel:")
print(result)

### 📖 Concept: Aggregate Functions

Common aggregate functions:
- **COUNT()**: Number of rows
- **SUM()**: Total of values
- **AVG()**: Average of values
- **MIN()**: Minimum value
- **MAX()**: Maximum value

In [ ]:
# Performance metrics by campaign
query = '''
SELECT 
    c.campaign_name,
    c.channel,
    COUNT(dp.performance_id) as days_active,
    SUM(dp.impressions) as total_impressions,
    SUM(dp.clicks) as total_clicks,
    SUM(dp.conversions) as total_conversions,
    SUM(dp.cost) as total_cost,
    SUM(dp.revenue) as total_revenue,
    ROUND(SUM(dp.revenue) / SUM(dp.cost), 2) as roas
FROM campaigns c
INNER JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
GROUP BY c.campaign_id, c.campaign_name, c.channel
ORDER BY roas DESC;
'''

result = pd.read_sql_query(query, conn)
print("Campaign Performance Summary:")
print(result)

### 📖 Concept: HAVING Clause

HAVING filters groups (WHERE filters rows before grouping).

In [ ]:
# Find campaigns with total cost > 10000
query = '''
SELECT 
    c.campaign_name,
    SUM(dp.cost) as total_cost,
    SUM(dp.conversions) as total_conversions,
    ROUND(SUM(dp.cost) / SUM(dp.conversions), 2) as cpa
FROM campaigns c
INNER JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
GROUP BY c.campaign_id, c.campaign_name
HAVING SUM(dp.cost) > 10000
ORDER BY cpa;
'''

result = pd.read_sql_query(query, conn)
print("High-Spend Campaigns (>$10,000):")
print(result)

### ✏️ Exercise 3: Aggregation Practice

In [ ]:
# YOUR CODE HERE
# 1. Calculate total conversions by channel
# Join campaigns and daily_performance
# Group by channel
# Show: channel, total conversions, avg conversions per day
# Order by total conversions descending

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print(result)

In [ ]:
# YOUR CODE HERE
# 2. Find customers with more than one conversion
# Use HAVING to filter
# Show: customer name, number of conversions, total value

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print(result)

### 🎯 Day 17 Mini-Project: Channel Performance Dashboard

Create a comprehensive channel-level analysis.

In [ ]:
# YOUR CODE HERE
# Create a channel performance report showing:
# - Channel
# - Number of campaigns
# - Total impressions, clicks, conversions
# - Total cost and revenue
# - Blended CTR (total clicks / total impressions)
# - Blended CVR (total conversions / total clicks)
# - Blended CPA (total cost / total conversions)
# - Blended ROAS (total revenue / total cost)
#
# Order by ROAS descending
# Format calculations to 4 decimal places for rates, 2 for currency

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Channel Performance Dashboard:")
# print(result)

### 🎓 Day 17 Key Takeaways

✅ GROUP BY aggregates data by categories  
✅ Aggregate functions: SUM, AVG, COUNT, MIN, MAX  
✅ HAVING filters grouped results  
✅ WHERE filters before grouping, HAVING after  
✅ Can calculate metrics in SELECT with aggregates  

**Next:** Tomorrow we'll learn subqueries for advanced filtering!

---

## 📅 Day 18-20: Advanced SQL Concepts

The following days cover:

**Day 18: Subqueries**
- Subqueries in WHERE clause
- Subqueries in FROM clause
- Correlated subqueries
- EXISTS and IN with subqueries

**Day 19: CASE Statements**
- Simple CASE expressions
- Searched CASE expressions
- Creating performance tiers
- Conditional aggregations

**Day 20: Window Functions Intro**
- ROW_NUMBER(), RANK(), DENSE_RANK()
- PARTITION BY
- Running totals with SUM() OVER()
- Moving averages

---

## 📅 Day 21: Week 3 Capstone - Marketing Database Queries (~60 min)

### Project: Comprehensive Marketing Analytics Report

**Scenario:**  
You're the Marketing Measurement Partner for an e-commerce company. Leadership needs a comprehensive SQL-based analysis of Q4 campaign performance to inform budget decisions for next quarter.

**Deliverables:**
1. Campaign performance rankings
2. Channel efficiency analysis
3. Customer segment analysis
4. Time-based trends
5. Attribution insights
6. Budget reallocation recommendations

Use everything you've learned: SELECT, WHERE, JOINs, GROUP BY, subqueries, CASE statements, and window functions.

### Challenge 1: Top Performing Campaigns

In [ ]:
# YOUR CODE HERE
# Find the top 5 campaigns by ROAS
# Include: campaign_name, channel, total_cost, total_revenue, ROAS, conversions
# Only include campaigns with at least $5000 in spend
# Use HAVING to filter

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Top 5 Campaigns by ROAS:")
# print(result)

### Challenge 2: Customer Lifetime Value Analysis

In [ ]:
# YOUR CODE HERE
# Analyze customers by segment
# Show: customer_segment, number of customers, avg conversions per customer,
#       avg conversion value, total revenue by segment
# Join customers and conversions
# Which segment is most valuable?

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Customer Segment Analysis:")
# print(result)

### Challenge 3: Channel Attribution Analysis

In [ ]:
# YOUR CODE HERE
# For each channel, show:
# - Number of campaigns
# - Number of unique customers acquired
# - Total conversion value attributed
# - Average conversion value
# - Customer acquisition cost (total cost / unique customers)
#
# Join campaigns, conversions, and customers
# Use COUNT(DISTINCT ...) for unique customers

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Channel Attribution Analysis:")
# print(result)

### Challenge 4: Performance Tier Classification

In [ ]:
# YOUR CODE HERE
# Use a CASE statement to classify each campaign:
# - 'Excellent' if ROAS >= 4.0
# - 'Good' if ROAS >= 3.0
# - 'Acceptable' if ROAS >= 2.0
# - 'Poor' if ROAS < 2.0
#
# Show: campaign_name, ROAS, performance_tier
# Then count how many campaigns in each tier

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Campaign Performance Tiers:")
# print(result)

### Challenge 5: Weekly Trend Analysis

In [ ]:
# YOUR CODE HERE
# Calculate weekly performance trends
# Group daily_performance by week
# Show: week, total_cost, total_revenue, weekly_roas
# Calculate week-over-week growth rate
#
# Hint: Use strftime('%W', date) to extract week number
# Bonus: Use a window function for week-over-week comparison

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Weekly Performance Trends:")
# print(result)

### Challenge 6: Executive Summary Query

In [ ]:
# YOUR CODE HERE
# Create a single comprehensive query that shows:
# - Overall portfolio metrics (total spend, revenue, ROAS, conversions)
# - Best performing channel
# - Number of active campaigns
# - Number of unique customers acquired
# - Average customer acquisition cost
#
# This should be a multi-part query or use subqueries

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Executive Summary:")
# print(result)

### 🎓 Week 3 Complete!

**Congratulations!** You've completed Week 3 of the Marketing Measurement Partner Academy.

**What You've Mastered:**
- ✅ SQL SELECT, WHERE, and filtering
- ✅ JOIN operations to combine tables
- ✅ GROUP BY and aggregations
- ✅ Subqueries for complex logic
- ✅ CASE statements for conditional logic
- ✅ Window functions for advanced analytics
- ✅ Real-world marketing database queries

**SQL Skills in Your Toolkit:**
You can now:
- Extract data from any relational database
- Perform dimensional analysis (by channel, campaign, time)
- Calculate complex marketing metrics
- Attribute conversions to campaigns
- Identify trends and opportunities

**Next Week Preview:**
Week 4 will cover **Advanced SQL** including:
- Advanced window functions (LAG, LEAD, NTILE)
- Common Table Expressions (CTEs)
- Query optimization
- Date/time operations
- String manipulation
- Complex marketing analytics queries

**Keep building your data skills!** 🚀

---